In [6]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
import os
from PIL import Image

from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import DataLoader, WeightedRandomSampler
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision import models
import torch._dynamo
torch._dynamo.config.suppress_errors = True

import pytorch_lightning as pl
from pytorch_lightning import LightningModule
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

from torchmetrics.classification import MulticlassAccuracy, F1Score
!uv pip install pytorch_optimizer
import pytorch_optimizer as optim1
import optuna

pl.seed_everything(42)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Using Python 3.11.13 environment at: /usr
Audited 1 package in 140ms


In [7]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        allowed_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif'}
        self.image_files = sorted([
        f for f in os.listdir(root_dir) if os.path.isfile(os.path.join(root_dir, f)) and os.path.splitext(f)[1].lower() in allowed_extensions
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        image_name = self.image_files[idx]
        return image, image_name

In [8]:
weights = models.EfficientNet_B0_Weights.IMAGENET1K_V1
auto_transforms = weights.transforms()
print(auto_transforms)

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


In [9]:
train= transforms.Compose([
             auto_transforms
])
val = transforms.Compose([
        auto_transforms
])
data_dir = '/kaggle/input/logika/Train/'

train_dataset = datasets.ImageFolder(root=f'{data_dir}Train', transform=train)
labels = train_dataset.targets
class_counts = torch.bincount(torch.tensor(labels))
print(f"Pemetaan kelas: {train_dataset.class_to_idx}")
print(f"Jumlah sampel per kelas: {class_counts}")
class_weights = 1.0 / class_counts.float()
print(f"Bobot untuk setiap kelas: {class_weights}")
weights_per_sample = class_weights[labels]
print(f"Panjang bobot per sampel: {len(weights_per_sample)}")
print("Contoh 5 bobot pertama:", weights_per_sample[:5])

sampler = WeightedRandomSampler(weights_per_sample, num_samples=len(weights_per_sample), replacement=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True, sampler=sampler)
print(f'Jumlah data test: {len(train_dataset)}')

Pemetaan kelas: {'balinese': 0, 'batak': 1, 'dayak': 2, 'javanese': 3, 'minangkabau': 4}
Jumlah sampel per kelas: tensor([776,  95,  69, 249, 563])
Bobot untuk setiap kelas: tensor([0.0013, 0.0105, 0.0145, 0.0040, 0.0018])
Panjang bobot per sampel: 1752
Contoh 5 bobot pertama: tensor([0.0013, 0.0013, 0.0013, 0.0013, 0.0013])
Jumlah data test: 1752


In [10]:
data_test_dir = '/kaggle/input/logika/Test/Test'

test_dataset = TestDataset(root_dir=data_test_dir, transform=val)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)
print(f'Jumlah data test: {len(test_dataset)}')

Jumlah data test: 444


In [11]:
feature = next(iter(test_loader))

In [12]:
# for i in feature:
#     print(f'{i}: {len(i)}')

In [13]:
label2cat, idxclass = train_dataset.class_to_idx, train_dataset.classes
label2cat

{'balinese': 0, 'batak': 1, 'dayak': 2, 'javanese': 3, 'minangkabau': 4}

## Arsitektur dan config

In [14]:
def conv_block(in_feature, out_feature, padding=1, stride=1,
             activation="relu", pool =True, maxpool=True, kernel_size=3,
             kernel_size_pool=2, pool_stride=2)-> list[nn.Sequential]:
    layers = [nn.Conv2d(in_feature, out_feature, kernel_size=kernel_size, padding=padding, stride=stride)]
    if activation == "relu":
        layers.append(nn.ReLU())
    elif activation == "leakyrelu":
        layers.append(nn.LeakyReLU())
    elif activation == "sigmoid":
        layers.append(nn.Sigmoid())
    elif activation == 'mish': layers.append(nn.Mish())
    elif activation == "tanh":
        layers.append(nn.Tanh())
    if pool:
        if maxpool:
            layers.append(nn.MaxPool2d(kernel_size=kernel_size_pool, stride=pool_stride))
        else:
            layers.append(nn.AvgPool2d(kernel_size=kernel_size_pool, stride=pool_stride))
    else:
        layers.append(nn.Identity())
    return nn.Sequential(*layers)


def linear_block(in_features, out_features, activation=None, dropout=0.0, batch_norm=None):
    layers = [nn.Linear(in_features, out_features)]
    if batch_norm:
        layers.append(nn.BatchNorm1d(out_features))
    if activation == 'relu':
        layers.append(nn.ReLU())
    elif activation == 'sigmoid':
        layers.append(nn.Sigmoid())
    elif activation == 'tanh':
        layers.append(nn.Tanh())
    elif activation == 'leakyrelu':
        layers.append(nn.LeakyReLU())
    elif activation == 'mish': layers.append(nn.Mish())
    elif activation == 'softmax':
        layers.append(nn.Softmax(dim=1))
    elif activation == 'elu':
        layers.append(nn.ELU())
    elif activation == 'selu':
        layers.append(nn.SELU())
    elif activation == 'silu':
        layers.append(nn.SiLU())
    elif activation == 'lsoftmax':
        layers.append(nn.LogSoftmax(dim=1))
    if dropout > 0.0:
        layers.append(nn.Dropout(dropout))
    return nn.Sequential(*layers)

In [15]:
class EfficientNet(nn.Module):
    def __init__(self, dropout=0.0, freeze=True):
        super().__init__()
        weights = models.EfficientNet_B0_Weights.IMAGENET1K_V1
        self.model = models.efficientnet_b0(weights=weights)
        if freeze:
            for param in self.model.parameters():
                param.requires_grad = False
        num_in_features = self.model.classifier[1].in_features
        self.model.classifier = nn.Sequential(
            linear_block(num_in_features, 256, activation='silu', dropout=dropout, batch_norm=True),
            linear_block(256, 128, activation='silu', dropout=dropout, batch_norm=True),
            linear_block(128, 5, activation=None)
        )
    def forward(self, X):
        return self.model(X)
        
class PL(LightningModule):
    def __init__(self, model, learning_rate, class_weights) -> None:
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.criterion = nn.CrossEntropyLoss(weight=class_weights)
        self.macroF1 = F1Score(num_classes=5, average='macro', task='multiclass')
    
    def forward(self, X):
        return self.model(X)
    
    def _common_step(self, batch, batch_idx):
        X, labels = batch
        outputs = self(X) 
        loss = self.criterion(outputs, labels)
        macrof1 = self.macroF1(outputs, labels)
        return loss, macrof1

    def training_step(self, batch, batch_idx):
        loss, macroF1 = self._common_step(batch, batch_idx)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_macrof1', macroF1, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, macroF1 = self._common_step(batch, batch_idx)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_macrof1', macroF1, on_epoch=True, prog_bar=True, logger=True)

    def test_step(self, batch, batch_idx):
        loss, macroF1 = self._common_step(batch, batch_idx)
        self.log('test_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('test_macrof1', macroF1, on_epoch=True, prog_bar=True, logger=True)

    def backward(self, loss, *args, **kwargs):
        loss.backward(create_graph=True)

    def configure_optimizers(self):
        optimizer = optim1.Lion(self.parameters(), lr=self.learning_rate)
        return optimizer

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        """
        Langkah prediksi untuk satu batch data test.
        """
        images, image_names = batch
        outputs = self.forward(images)
        _, predicted_labels = torch.max(outputs, 1)
        return {"image_names": image_names, "preds": predicted_labels}

In [16]:
if torch.cuda.is_available():
    accelerator_type = 'gpu'
    devices_to_use = 1
else:
    accelerator_type = 'cpu'
    devices_to_use = 'auto'

checkpoint_callback = ModelCheckpoint(
    monitor='train_macrof1',
    dirpath='checkpoints/',
    filename='logikaui-{epoch:02d}-{train_macrof1:.2f}',
    save_top_k=1,
    mode='max'
)
early_stopping = EarlyStopping(
    monitor='train_loss',
    patience=10,
    mode='min',
)
lr_monitor_callback = LearningRateMonitor(logging_interval='epoch')

trainer1 = pl.Trainer(
    max_epochs=300,
    callbacks=[checkpoint_callback, early_stopping, lr_monitor_callback],
    logger=TensorBoardLogger("tb_logs", name="simple_model_experiment"),
    accelerator=accelerator_type,
    devices=devices_to_use,
    log_every_n_steps=10,
    deterministic=True,
    precision='16-mixed'

)

## Train

In [17]:
class_weights

tensor([0.0013, 0.0105, 0.0145, 0.0040, 0.0018])

In [18]:
model = PL(EfficientNet(dropout=0.1, freeze=True), learning_rate=5e-4, class_weights=class_weights)
# model = torch.compile(model) 

In [19]:
trainer1.fit(model, train_loader, val_dataloaders=None)

2025-09-25 04:31:17.483291: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758774677.504508     151 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758774677.512198     151 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Training: |          | 0/? [00:00<?, ?it/s]

NameError: name 'exit' is not defined

In [20]:
if __name__ == "__main__":
    preds = trainer1.predict(model, test_loader, ckpt_path='best')
    print(preds)

Predicting: |          | 0/? [00:00<?, ?it/s]

[{'image_names': ['Test_001.jpg', 'Test_002.jpg', 'Test_003.jpg', 'Test_004.jpg', 'Test_005.jpg', 'Test_006.jpg', 'Test_007.jpg', 'Test_008.jpg', 'Test_009.jpg', 'Test_010.jpg', 'Test_011.jpg', 'Test_012.jpg', 'Test_013.jpg', 'Test_014.jpg', 'Test_015.jpg', 'Test_016.jpg'], 'preds': tensor([0, 2, 4, 0, 1, 2, 0, 0, 3, 4, 3, 2, 1, 0, 2, 3])}, {'image_names': ['Test_017.jpg', 'Test_018.jpg', 'Test_019.jpg', 'Test_020.jpg', 'Test_021.jpg', 'Test_022.jpg', 'Test_023.jpg', 'Test_024.jpg', 'Test_025.jpg', 'Test_026.jpg', 'Test_027.jpg', 'Test_028.jpg', 'Test_029.jpg', 'Test_030.jpg', 'Test_031.jpg', 'Test_032.jpg'], 'preds': tensor([1, 3, 2, 2, 0, 1, 0, 0, 2, 0, 0, 3, 4, 2, 2, 2])}, {'image_names': ['Test_033.jpg', 'Test_034.jpg', 'Test_035.jpg', 'Test_036.jpg', 'Test_037.jpg', 'Test_038.jpg', 'Test_039.jpg', 'Test_040.jpg', 'Test_041.jpg', 'Test_042.jpg', 'Test_043.jpg', 'Test_044.jpg', 'Test_045.jpg', 'Test_046.jpg', 'Test_047.jpg', 'Test_048.jpg'], 'preds': tensor([0, 3, 0, 1, 0, 4, 0, 3, 

In [21]:
predictions = []
for batch_result in preds:
    # batch_result adalah dictionary yang kita return dari predict_step
    image_names = batch_result['image_names']
    preds = batch_result['preds'].cpu().numpy() # Pindahkan ke CPU
    
    for name, label in zip(image_names, preds):
        predictions.append({
            'id': name,      # Nama kolom sesuai standar Kaggle
            'style': label   # Nama kolom sesuai standar Kaggle
        })
predictions

[{'id': 'Test_001.jpg', 'style': 0},
 {'id': 'Test_002.jpg', 'style': 2},
 {'id': 'Test_003.jpg', 'style': 4},
 {'id': 'Test_004.jpg', 'style': 0},
 {'id': 'Test_005.jpg', 'style': 1},
 {'id': 'Test_006.jpg', 'style': 2},
 {'id': 'Test_007.jpg', 'style': 0},
 {'id': 'Test_008.jpg', 'style': 0},
 {'id': 'Test_009.jpg', 'style': 3},
 {'id': 'Test_010.jpg', 'style': 4},
 {'id': 'Test_011.jpg', 'style': 3},
 {'id': 'Test_012.jpg', 'style': 2},
 {'id': 'Test_013.jpg', 'style': 1},
 {'id': 'Test_014.jpg', 'style': 0},
 {'id': 'Test_015.jpg', 'style': 2},
 {'id': 'Test_016.jpg', 'style': 3},
 {'id': 'Test_017.jpg', 'style': 1},
 {'id': 'Test_018.jpg', 'style': 3},
 {'id': 'Test_019.jpg', 'style': 2},
 {'id': 'Test_020.jpg', 'style': 2},
 {'id': 'Test_021.jpg', 'style': 0},
 {'id': 'Test_022.jpg', 'style': 1},
 {'id': 'Test_023.jpg', 'style': 0},
 {'id': 'Test_024.jpg', 'style': 0},
 {'id': 'Test_025.jpg', 'style': 2},
 {'id': 'Test_026.jpg', 'style': 0},
 {'id': 'Test_027.jpg', 'style': 0},
 

In [22]:
submission_df = pd.DataFrame(predictions)

In [23]:
class_mapping = {'balinese': 0, 'batak': 1, 'dayak': 2, 'javanese': 3, 'minangkabau': 4}
idx_to_class = {v: k for k, v in class_mapping.items()}
# Cek tipe data dari kolom 'style'. Kemungkinan besar hasilnya 'object' (string).
print(f"Tipe data kolom 'style': {submission_df['style'].dtype}")
submission_df['id'] = submission_df['id'].str.split('.').str[0]

# Lihat nilai-nilai unik di dalam kolom tersebut.
# Perhatikan apakah ada tanda kutip, yang menandakan string.
print(f"Nilai unik di kolom 'style': {submission_df['style'].unique()}")
submission_df['style'] = submission_df['style'].map(idx_to_class)
submission_df.head()

Tipe data kolom 'style': int64
Nilai unik di kolom 'style': [0 2 4 1 3]


,id,style
0,Test_001,balinese
1,Test_002,dayak
2,Test_003,minangkabau
3,Test_004,balinese
4,Test_005,batak


In [24]:
submission_df.to_csv('submission.csv', index=False)